In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

Flag = 1 => Pass \
Flag = 3 => Suspect \
Flag = 4 => Fail

# Short Term Time Series Tests

**Test 9: ST Time Series Gap Test.**


First, we read the data

In [ ]:
calm_weather = pd.read_csv("../input/wavedatav1/M2_M3_M4_M5_M6 Apr2020_May2020 - Calm WeatherV1.csv")

We then calculate the max number of consecutive missing points for each variable.
(Here only hm0 is done for now)

In [ ]:
#only hm0 is dealt with here, need a way of running through all variables.
hm0_missing = calm_weather.hm0.isnull().astype(int).groupby(calm_weather.hm0.notnull().astype(int).cumsum()).cumsum()
hm0_C2 = max(hm0_missing)
hm0_C2

We then run our test against N, set by the operator and set the flag for each variable in the time series.

In [ ]:
def test_9(C2, N):
    if C2>=N:
        flag = 4
    else:
        flag = 1
    print(flag)
N=4
test_9(hm0_C2,N)

**Test 10: ST Time Series Spike Test.**

Read the data - only use a small amount for this test, say 1 day

In [ ]:
calm_weather_day1 = pd.read_csv("../input/wavedata2/CalmWeather Day1.csv")

Calculate means and standard deviations for each variable - try airpressure

In [ ]:
airp_mean = calm_weather_day1["airpressure"].mean()
airp_sd = calm_weather_day1["airpressure"].std()
M=4

Perform the test: 
The spike test checks for spikes in a time series. Spikes are defined as points more than M times the standard 
deviation (SD) from the mean—a WMO standard of 4 * SD is often used. After the ST time series is 
received, the mean (MEAN) and standard deviation must be determined. Counters M1 and M2 are set to 0. 
Once a spike has been identified, the spike is replaced with the average (AVG) of the previous point (n - 1) 
and the following point (n + 1). Alternative interpolation methods such as a spline fit may also be used. The 
counter, M1, is incremented as spikes are identified. The algorithm should iterate over the time series multiple 
(P) times, re-computing the mean and standard deviation for each iteration. After the Pth iteration, a final 
spike count, M2, is run. The counters M1 and M2 are compared to the number of spikes allowed. The time 
series is rejected if it contains too many spikes (generally set to N% of all points) or if spikes remain after P
iterations (M2 > 0).

First, check for spikes

In [ ]:
for i in range(len(calm_weather_day1.airpressure)):
    print (abs(calm_weather_day1.airpressure[i]-airp_mean)>M*airp_sd)

If observation $i$ is a spike, replace $i$ with $((i-1)+(i+1))/2$

Repeat the above algorithm P times(operator defined) to smooth out all spikes. If the amount of spikes is too great(>some value M2), fail the time series

**Test 11: ST Time Series Range Test**

The range test checks that the values (e.g., pressure, AST, u, v) of the time series fall within limits defined by 
the operator. The operator should at least define the instrument range for these tests. Regional or 
seasonal/climate ranges may also be provided. If the instrument range is exceeded, data should be flagged as 
failed.

The operator defines the instrument minimum (IMIN) and instrument maximum (IMAX) and may also 
define the local minimum (LMIN) and local maximum (LMAX). The local maximum and minimum may be 
location-, season-, and/or sensor-dependent.
TSVAL is the value of the time series at point, $i$. 

In [ ]:
IMIN = 0
IMAX = 1084.8
LMIN = 900
LMAX = 1080
def test_11_a():
    for i in range(len(calm_weather_day1.airpressure)):
        print (calm_weather_day1.airpressure[i]>IMAX or calm_weather_day1.airpressure[i]<IMIN)

In [ ]:
def test_11_b():
    for i in range(len(calm_weather_day1.airpressure)):
        print (calm_weather_day1.airpressure[i]>LMAX or calm_weather_day1.airpressure[i]<LMIN)

In [ ]:
test_11_a()

**Test 12: ST Time Segment Shift Test**

The time series is broken into n segments m points long. Segment means are computed for each of the n
segments. Each segment mean is compared to neighboring segments. If the difference in the means of two 
consecutive segments exceeds P, the ST time series data are rejected. The operator defines n segments, m
points, and P

The operator determines the number of segments (n) to be compared in the time series and the length of 
each segment (m) to be compared in the time series. Then, m or n can be computed by the other in 
conjunction with the length of the entire time series. The length of m should be consistent with statistical 
best practices. The operator also defines the mean shift (P) that is allowed in the time series.
A mean value (MEAN [n]) is computed for each of the n segments. The means of consecutive segment are 
then compared. If the differences of the means exceed the allowed mean shift (P) provided by the user, the 
entire time series is failed

In [ ]:
#split data into 3 groups of 8
m = 4
n = 8
P = 0.2
for i in range(1,m):
    print(calm_weather_day1.iloc[:n*(i),7].mean()-calm_weather_day1.iloc[:n*(i+1),7].mean())
    #print(abs(calm_weather_day1.iloc[:n*i,7].mean()-calm_weather_day1.iloc[:n*(i+1),7].mean())>P)

**Test 13: ST Time Series Acceleration Test**

The in-situ systems that collect these time series data can accumulate accelerations in all directions from 
multiple sensors. Any acceleration that exceeds a practical value should be replaced by an 
interpolated/extrapolated value.

Acceleration (a) is defined as the product of M and G, M is an operator-defined value, and G is the 
gravitational acceleration (9.80 m/s2
).
Any acceleration values exceeding M*G are replaced with operator-defined interpolated/extrapolated values. 
A counter, M5, is initially set to 0 and is incremented by one as each point is replaced. The operator defines up 
to N points that may be replaced

In [ ]:
#Need accelerometer data to do this check
#Only derived data is what we recieve

# Long Term Time Series Tests

**Test 14: LT Time Series Check Ratio or Check Factor Test**

In [ ]:
#Need spectral data to check this

# LT - Long Term time series tests

**Bulk Wave Parameters**\
Spectral Significant Wave Height (Hm0): hm0\
Peak Wave Period (TP): tm02\
Mean Wave Direction (qd): mdir \
To reference a dataset and a wave parameter to run a test on, type "test('dataset'.'bulk wave parameter')"\
Example:\
To perform test 15 on the peak wave period(hm0) for the stormy weather period, run:\
    test_15(stormy_weather.hm0)

Read the files...

In [ ]:
stormy_weather = pd.read_csv("../input/wavedata3/M2_M3_M4_M5_M6 Nov2013_Mar2014 - StormyV1.csv")

In [ ]:
calm_weather = pd.read_csv("../input/wavedatav1/M2_M3_M4_M5_M6 Apr2020_May2020 - Calm WeatherV1.csv")

**Test 15: LT Time Series Mean and Standard Deviation**

This test applies to all in-situ wave measuring systems and most bulk wave parameters (few operators will test 
wave spread). Series mean values are compared to thresholds defined by the operator. Thresholds are 
determined by an operator-defined mean plus an operator-defined allowable variance from the mean.

Check that TSVAL value is within limits defined by the operator. Operator defines the period over which the 
mean and standard deviation are calculated and the number of allowable standard deviations (N).

This check is for checking if values are within plausible standard deviations from the mean

In [ ]:
def test_15(df,N):
    WVHGT_mean = df.mean()
    WVHGT_sd = df.std()
    for i in range(len(df)):
        if df[i]<(WVHGT_mean-(N*WVHGT_sd)) or df[i]>(WVHGT_mean+(N*WVHGT_sd)):
            print(i)

In [ ]:
test_15(stormy_weather.hm0,4);#N=4 find the mosextreme 0.01% of values)

In [ ]:
test_15(stormy_weather.tm02,4)

In [ ]:
test_15(stormy_weather.mdir,4)

In [ ]:
test_15(calm_weather.hm0,4)

In [ ]:
test_15(calm_weather.tm02,4)

In [ ]:
test_15(calm_weather.mdir,4)

**Test 16 - Flatline test**

This test checks for invariate observations and can be applied to all bulk wave parameters that are reported.

In [ ]:
epsilon = 0.001
def test_16(df,epsilon):
    print("The following observations have failed:")
    #make sure all items in list are floats, not strings
    [float(i) for i in df]
    for i in range(len(df)-5):
        diff_1 = abs(df[i]-df[i+1])
        diff_2 = abs(df[i+1]-df[i+2])
        diff_3 = abs(df[i+2]-df[i+3])
        diff_4 = abs(df[i+3]-df[i+4])
        diff_5 = abs(df[i+4]-df[i+5])
        if diff_1<epsilon and diff_2<epsilon and diff_3<epsilon and diff_4<epsilon and diff_5<epsilon and df[i]!=0:
            print(i,df[i],df[i+1],df[i+2],df[i+3],df[i+4],df[i+5])
            
            
#Also need to code a suspect test
    

In [ ]:
test_16(stormy_weather.hm0,0.001)

In [ ]:
test_16(stormy_weather.tm02,0.001)

In [ ]:
test_16(stormy_weather.mdir,0.001)

In [ ]:
test_16(calm_weather.hm0,0.001)

In [ ]:
test_16(calm_weather.tm02,0.001)

In [ ]:
test_16(calm_weather.mdir,0.001)

As we can see from the above results, there are quite a few observations that fail the flatline test, both in calm and stormy conditions. \
What do we do with this data?


**Test 17 - Relevant to Spectral data only**

**LT Time Series Low-Frequency Energy (Test 18) -Wave energy??**

**LT Time Series Bulk Wave Parameters Max/Min/Acceptable Range (Test 19)**

A test for maximum, minimum, and acceptable range for bulk wave parameters

The operator should establish maximum and minimum values for the bulk wave parameters:wave height (WVHGT), period (WVPD), direction (WVDIR), and spreading (WVSP) (if provided). If the wave height fails this test, then no bulk wave parameters should be released. Otherwise, suspect flags are set.Operatorsupplies minimum wave height (MINWH), maximum wave height (MAXWH), minimum wave period (MINWP), maximum wave period (MAXWP), minimum spreading value (MINSV), and maximum spreading value (MAXSV).

In [ ]:
def test_19(df):
    MINWH = 0
    MAXWH = 30
    MINWP = 0
    MAXWP = 15 #This is a guess, not sure what the largest wave period should be
    MINWD = 0.0
    MAXWD = 360
    WVHGT_max = max(df.hm0)
    WVHGT_min = min(df.hm0)
    WVPD_max = max(df.tm02)
    WVPD_min = min(df.tm02)
    WVDIR_max = max(df.mdir)
    WVDIR_min = min(df.mdir)
    if WVHGT_min < MINWH or WVHGT_max > MAXWH:
        flag = 4
    elif WVPD_min <MINWP or WVPD_max >MAXWP:
        flag = 3
    elif WVDIR_min <0.0 or WVDIR_max >360:
        flag = 2
    else:
        flag = 1
    return flag

In [ ]:
test_19(stormy_weather)

In [ ]:
test_19(calm_weather)

**LT Time Series Rate of Change (Test 20)**

This test evaluates the rate of change with time, i.e., a maximum limit is placed on the rate of change between successive data points, or specific data pointsat defined times. It can also be considered a spiketest.

A test for acceptable rate of change: \
This test is applied only to wave heights, average wave periods, and parameters that are a result of expected changes due to winds and constitute an integration of the whole or relevant portions of the spectrum (e.g., wind waves).The test described here uses significant wave height as an example.The operator selects a threshold value, MAXHSDIFF, and the two most recent data points Hs(n) and Hs(n-1) are checked to see if the rate of change is exceeded.

In [ ]:
delta = 10 # What is a good value for delta here?
def test_20(df,delta):
    print("The following observations have failed:")
    #make sure all items in list are floats, not strings
    [float(i) for i in df]
    for i in range(len(df)-1):
        diff = abs(df[i]-df[i+1])
        if diff>delta:
            print(i,df[i],i+1,df[i+1])

In [ ]:
test_20(stormy_weather.hm0,5)

In [ ]:
test_20(stormy_weather.tm02,5)

In [ ]:
test_20(stormy_weather.mdir,270)

In [ ]:
test_20(calm_weather.hm0,5)

In [ ]:
test_20(calm_weather.tm02,5)

In [ ]:
test_20(calm_weather.mdir,270)

**Neighbor Check (Test 21) -Suggested**

This check has the potential to be the most useful test when a nearby second sensor is determined to have a similar response.Ideally, redundant wavesensors utilizing different technology would be co-located and alternately serviced at different intervals. This close neighbor would provide the ultimate QC check, but cost prohibits such a deployment in most cases.However, there are very few instances where a second sensor is sufficiently proximate to provide a useful QC check. Just a few hundred meters of horizontal separationcan yield greatly different results. Only an experienced operator can determine the extent to which adjacent waves sensors would agree. Nevertheless, the test should not be overlooked where it may have application.This test is similar to the LT time series wave parameters max/min/acceptable range (test 19),where theagreement is constrained to matching the second wave sensorwithin allowable difference (Delta). The selected thresholds depend entirely upon the relationship between thetwo sensors as determined by the local knowledge of the operator.In the instructions and examples below, bulk parameter data from one site (W1) are compared to a second site (W2).

In [ ]:
def test_21(df1,df2,delta):
    flag = 1
    for i in range(len(df1)-1):
        diff = abs(df1[i]-df2[i])
        if diff>delta:
            flag = 3
            print(i,df1[i],df2[i])
    return flag


In [ ]:
test_21(calm_weather.hm0,stormy_weather.hm0,10)#doesnt really make sense seeing as this test only applies to measurements taken 
                                               #at the same time in close locations, but the test should run fine once we have that data